In [1]:
import string
import numpy as np
import scipy as sc
import pandas as pd
import pickle
import csv
import matplotlib.pyplot as plt
from sklearn import *
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import *
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.linear_model import *
from sklearn.ensemble import *
from sklearn.metrics import *
from sklearn.externals import *
from sklearn.utils import shuffle
from sklearn.naive_bayes import *
import tensorflow as tf
from tensorflow import keras
from nltk.corpus import stopwords
from nltk import *
from textblob import TextBlob, Word
import subprocess
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.preprocessing import text

from helpers import preprocessData, getFeatures, create_csv_submission

In [2]:
## Global Variables

## Hyper-Parameters
CLASSIFIER = "logistic"
CLEAN = False ## determines if we clean the dataset using the cleaning function we have defined

## File Paths
DATA_POS = 'data/train_pos_full.txt'
DATA_NEG = 'data/train_neg_full.txt'
VOCAB = 'data/vocab.pkl'
TEST = 'data/test_data.txt'
EMBEDIINGS = 'tools/embeddings.npy'
    
PREDICTION_PATH = "M%s_c%spred.csv" % (CLASSIFIER, CLEAN)

## If True, train the model on this notebook - else load model from pickle to generate predictions.
TRAIN_MODEL = True

# If True, use validation data
USE_VALID = True

### Data preprocessing

In [3]:
if CLASSIFIER in ["logistic", "random-forest"]:
    # Loading features from embeddings
    pos_data = getFeatures(DATA_POS)
    neg_data = getFeatures(DATA_NEG)
    test_data = getFeatures(TEST)

if CLASSIFIER in ["svm", "naive-bayes"]:    
    # Loading data from files and convert to numpy arrays
    pos_data = pd.read_csv(DATA_POS, header=None, delimiter="\n", names=["tweets"]).tweets.values
    neg_data = pd.read_csv(DATA_NEG, header=None, delimiter="\n", names=["tweets"]).tweets.values
    test_data = pd.read_csv(TEST, header=None, delimiter="\n", names=["tweets"]).tweets.values

In [4]:
# Prepare data before training
train_data, train_labels = preprocessData(pos_data, neg_data)

if USE_VALID:
    ## Split training and testing data into training and validation set
    train_data, validation_data, train_labels, validation_labels = train_test_split(train_data, train_labels)

### Training model

In [5]:
if CLASSIFIER == "logistic":
    clf = LogisticRegression()
    clf.fit(train_data, train_labels)
    
if CLASSIFIER == "random-forest":
    clf = RandomForestClassifier(n_estimators=100)
    clf.fit(train_data, train_labels)
    
if CLASSIFIER == "svm":
    clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None))])
    clf.fit(train_data, train_labels)  
    
if CLASSIFIER == "naive-bayes":
    clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
    clf.fit(train_data, train_labels)

/home/mikanikos/anaconda3/envs/tf/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [6]:
# Evaluate model

if USE_VALID:
    y_pred_val = clf.predict(validation_data)
    print(accuracy_score(validation_labels, y_pred_val))

# Generate predictions
y_pred = clf.predict(test_data)

0.6069264


### Prepare submission

In [ ]:
y_pred[np.where(y_pred <= 0.5)] = 0
y_pred[np.where(y_pred > 0.5)] = 1
create_csv_submission(range(1,10001), y_pred, PREDICTION_PATH, False)
print("Predict at", PREDICTION_PATH)